In [6]:
import numpy as np                                                  # to preform numerical operations
from sklearn import datasets                                        # to import required dataset
from sklearn import model_selection                                 # to split data into training and testing data
from nltk.corpus import stopwords                                   # for removing stopwords
import itertools                                                    # for slicing vocabulary dictionary into required length
import  pandas as pd                                                # for dataset manipulation
from sklearn.metrics import classification_report, confusion_matrix # for getting info of result precision and accuracy
from nltk.tokenize import word_tokenize                             # for tokenizing  
import math as ma

In [7]:
stop_words=set(stopwords.words('english'))                                            # get the stopwords 
news=datasets.fetch_20newsgroups()                                                    # load data into news 
x_train,x_test,y_train,y_test=model_selection.train_test_split(news.data,news.target,test_size=0.1) # split data into test and train    

In [8]:
print(stop_words)                                                   # Just checking what are the stopwords

{'himself', "should've", 'her', "you'll", 'doing', 'most', 'aren', 'why', 'any', 'couldn', 'then', 'your', 'his', 's', 'are', 'they', 'own', 'weren', 'you', 'again', 'just', 'to', 'out', 'hadn', 'but', 'won', 'mustn', 'y', "wouldn't", 'there', "it's", "haven't", 'during', 'by', 'when', "couldn't", 'shouldn', 'very', 'than', 'had', 'mightn', 'under', 'needn', 'not', "that'll", 'ours', 'below', "mustn't", 'she', 'on', 'were', 'so', 't', 'while', 'which', 'll', 'from', 'should', 'only', "she's", 'other', 'an', 're', 'theirs', 'more', 'now', 'having', 'hasn', 'after', 'does', 'who', 'this', 'its', "isn't", 'wasn', 'yourself', 'because', 'didn', 'wouldn', 'of', 'those', 'm', 'same', 'all', 'i', 'them', 'o', 'our', 'these', 'up', "you'd", 'through', 'such', 'further', "won't", 'hers', 'was', 'before', 'itself', 'him', 'off', "doesn't", 'yourselves', "you've", 'or', "didn't", 'been', 'isn', 'ourselves', 'being', 'will', 'don', 'did', 'how', 'into', 'ma', 'yours', 'haven', 'over', 'few', 'can'

In [9]:
#import nltk
#nltk.download('punkt')                                             # needed to be done first time
d={}                                                                # d is going to be our vocabulary dictionary
for i in range(len(x_train)):                                       # this function filters stopwords and creates vocabulary
    data=x_train[i]                                                 # taken one row at a time of training data
    words=word_tokenize(data)                                       # tokenize the data
    filtered_sentence=[ele for ele in words if ele not in stop_words] # get filtered sentence
    for ele in filtered_sentence:                                   # create vocabulary from filtered sentence    
        d[ele]=d.get(ele,0)+1                                         
new_dict={}                                                         # this will be our final vocabulary dictionary
for key,value in sorted(d.items(), key=lambda kv: kv[1],reverse=True): # put the key,value pair from d in decreasing order of value
    new_dict[key]=value
new_dict=dict(itertools.islice(new_dict.items(),6000))             # Take the top 6000 elements into final vocabulary
features=[]
for i in new_dict:
    features.append(i)

In [12]:
print(new_dict)                                                     # just checking final created vocabulary dictionary
print(features)

{'>': 164218, ',': 145003, '.': 129758, '--': 105337, ':': 102469, ')': 64656, '(': 62716, "'AX": 51401, '@': 50089, 'I': 43898, '?': 32365, "''": 29113, '<': 23667, '!': 22424, '``': 21799, '%': 17280, '$': 17159, '|': 17078, ';': 16353, '#': 15969, "'s": 14569, ']': 14367, 'The': 14083, '-': 13558, '[': 13211, "n't": 13024, "'": 12802, '&': 12622, 'From': 10958, '`': 10760, 'Subject': 10541, '...': 10367, 'Lines': 10213, 'Organization': 9850, 'In': 8464, 'would': 8164, 'Re': 7131, 'writes': 7047, 'one': 6912, '1': 6088, 'article': 6009, 'It': 5386, '*': 5051, 'people': 5034, 'like': 5024, 'University': 4963, 'If': 4779, '2': 4733, 'X': 4681, 'know': 4585, 'A': 4348, 'M': 4343, 'get': 4266, 'This': 3980, 'think': 3929, 'MAX': 3706, 'use': 3617, '0': 3399, 'time': 3342, "'m": 3249, 'Q': 3156, 'could': 3129, 'also': 3065, '3': 3029, 'good': 2899, 'You': 2874, '7': 2859, '=': 2740, 'What': 2663, '4': 2658, 'even': 2565, 'R': 2548, 'way': 2542, '/': 2498, 'And': 2492, '+': 2485, 'may': 24

In [13]:
def updatedRow(words,features):                                   
    lst=np.array([0]*6000)                                         # Initialise the lst to be np.zeros with dimensions 1 X 6000 
    for ele in words:                                              # i will help us iterate our lst
        if ele in features:                             
            lst[features.index(ele)]=words.count(ele)
    return lst

def updateXTrain(x_train,features):
    x_train_updated=pd.DataFrame(np.zeros((len(x_train),6000)))     # Intialise the np.zeros 2-D array to x_train_updated
    x_train_updated.columns=features                                # Add columns to the x_train_updated
    for i in range(len(x_train)):                                   # Now operate on each row
        curr_x=x_train[i]                                           # curr_x contains the row i content
        words=word_tokenize(curr_x)                                 # extract all words of curr_x into words
        x_train_updated.iloc[i,:]=updatedRow(words,features)        # replace row i with the updateRow() function's list
    return x_train_updated                                          # return x_train_updated

x_train_updated=updateXTrain(x_train,features)                      # get updated x_train_updated
x_train_updated.columns                                             # just checking the columns

Index(['>', ',', '.', '--', ':', ')', '(', ''AX', '@', 'I',
       ...
       'Barbara', 'CX', 'Rockies', 'compuserve.com', 'Motor', 'SB', 'NB',
       'fiction', 'Moreover', '25.00'],
      dtype='object', length=6000)

In [14]:
def fit(X_train, Y_train):                                          # This function is used to fit training data into our model
    result = {}                                                     # This dictionary is going to be useful in later calculations
    class_values = set(Y_train) 
    for current_class in class_values:                              # We create keys for all the possible classes
        result[current_class] = {}                                  # We create a dictionary as value for each key itself
        result["total"] = len(Y_train)                              # It gives total element present in our dictionary
        current_class_rows = (Y_train == current_class)             # Obtain rows for the current class
        X_train_current = X_train[current_class_rows]               # Filter the x_train for current class
        Y_train_current = Y_train[current_class_rows]               # Filter the y_train for current class
        num_features = X_train.shape[1]                             
        result[current_class]["count"] = len(Y_train_current)       # It gives total number of features of our data
        a=0                                                         # To get total number of a particular feature 
        for j in range(num_features):                               # traverse each feature
            result[current_class][j]=X_train_current[:,j].sum()     # Get total number of current feature
            a+=result[current_class][j]                             # Increment a by total number of current feature
        result[current_class]['total']=a                            # Assign a, will be used in later calculations
    return result                                                   # Return result


x_train_updated=np.array(x_train_updated)                           # Change pandas dataframe to numpy array
d=fit(x_train_updated,y_train)                                      # Fit ur trainig data 

In [15]:
def probability(dictionary, x, current_class): 
    output = ma.log(dictionary[current_class]["total"]) - ma.log(dictionary["total"])
    num_features = len(dictionary[current_class].keys()) - 2
    for j in range( num_features ):
        if x[j]==0:
            continue
        count_current_class_with_value_j = dictionary[current_class][j]+1
        count_current_class = dictionary[current_class]['total']+num_features
        current_j_probablity = ma.log(count_current_class_with_value_j) - ma.log(count_current_class)
        output = output + current_j_probablity
    return output

def doSinglePrediction(x,dictionary):                                  # Function to predict class for a single row
    classes = dictionary.keys()                                        # Get all possible classes
    best_p = -100                                                      # Initialise best probablity & class to some -ve number
    best_class = -100 
    first_run = True                                                   # Running for first time = True
    for current_class in classes:                                      # Iterate over each possible class
        if (current_class == "total"):                                 # Ignore 'total' key
            continue
        p_current_class = probability(dictionary, x, current_class)    # Get probablity for x belonging to current class
        if (first_run or p_current_class > best_p):                    # If this is greatest probablity till now, change the
            best_p = p_current_class                                    # value of greatest probability & predicted class
            best_class = current_class
        first_run = False                                              # First run complete
    return best_class                                                  # Return the predicted class out of all classes

def y_predict(x_test,d,features):                                       # Function to predict the output values for test data
    y_pred=np.zeros(len(x_test))                                        # We will use this numpy array to store predictions
    x_test_updated=updateXTrain(x_test,features)                        # Update x_test so that our model can operate on it
    x_test_updated=np.array(x_test_updated)                             # Change pandas dataframe to numpy array
    for i in range(len(x_test)):                                                                                                        # Extract one-one row from data
        curr_x=x_test_updated[i,:]                                      # Extract one-one row 
        y_pred[i]=doSinglePrediction(curr_x,d)                          # Predict class for current row, store the prediction
    return y_pred                                                       # Return predictions


In [16]:
y_pred=y_predict(x_test,d,features)                                     # Call function to predict classes for x_test
print(classification_report(y_test,y_pred))                             # Print classification report
print(confusion_matrix(y_test,y_pred))                                  # Print confusion matrix


              precision    recall  f1-score   support

           0       0.82      0.78      0.80        41
           1       0.51      0.77      0.62        52
           2       1.00      0.04      0.08        47
           3       0.50      0.73      0.59        52
           4       0.64      0.95      0.76        56
           5       0.98      0.59      0.73        68
           6       0.76      0.88      0.81        64
           7       0.78      0.85      0.81        61
           8       0.80      0.85      0.83        62
           9       0.90      0.87      0.89        71
          10       1.00      0.84      0.91        57
          11       0.96      0.93      0.95        59
          12       0.68      0.77      0.72        44
          13       0.94      0.80      0.86        64
          14       0.90      0.91      0.91        58
          15       0.81      0.91      0.86        67
          16       0.95      0.90      0.92        58
          17       0.97    

In [17]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

0.7985865724381626


In [18]:
print('\n\n\t\tCOMPARING WITH LIBRARY IMPLEMENTATION')      # Compare built model with the sklearn's inbuilt model

from sklearn.naive_bayes import MultinomialNB               # Import MultinomialDB from sklearn
clf = MultinomialNB()                                       # Create object of MultinomialDB
clf.fit(x_train_updated, y_train)                           # Fit the training data
x_test_updated=updateXTrain(x_test,features)                # Update x_test so that model can operate on it
x_test_updated=np.array(x_test_updated)                     # Convert to np array
y_pred = clf.predict(x_test_updated)                        # Do predictions
print(classification_report(y_test,y_pred))                 # Print classification report
print(confusion_matrix(y_test,y_pred))                      # Print confusion matrix
print(accuracy_score(y_test,y_pred))



		COMPARING WITH LIBRARY IMPLEMENTATION
              precision    recall  f1-score   support

           0       0.76      0.78      0.77        41
           1       0.60      0.79      0.68        52
           2       0.67      0.04      0.08        47
           3       0.55      0.81      0.65        52
           4       0.65      0.86      0.74        56
           5       0.76      0.69      0.72        68
           6       0.67      0.86      0.75        64
           7       0.86      0.82      0.84        61
           8       0.74      0.89      0.81        62
           9       0.87      0.87      0.87        71
          10       1.00      0.81      0.89        57
          11       0.96      0.92      0.94        59
          12       0.76      0.73      0.74        44
          13       0.95      0.83      0.88        64
          14       0.96      0.86      0.91        58
          15       0.90      0.91      0.90        67
          16       0.74      0.91      

On comparing above results we can see that our implementation is performing better than the library implementation of Naive Bayes.